In [1]:
from transformers.utils import is_cython_available
import os
import numpy as np

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

print("Cython is installed:", is_cython_available())

Cython is installed: True


In [2]:
import pandas as pd

edges = pd.read_csv("Edge/totaledge_edge_breast-cancer.csv")
nodes = pd.read_csv("Node/totalnode_brca_tcga.csv")

In [3]:
nodes = nodes.drop(nodes[nodes["cancer_type"] != "Breast Cancer"].index)
# nodes
nodes = nodes[:32] #################### data 양 1개로 줄이기 ####################
nodes = nodes.reset_index(drop=True)
nodes["cancer_type_detailed"].value_counts()

graph = [[], []]
cnt = 0
for idx, row in edges.iterrows():
  col_list = nodes.columns.tolist()
  if(row["source"] not in col_list or row["target"] not in col_list):
    print("something wrong")
    cnt += 1
    continue
  if(row["weight"]): # if weight != 0
    graph[0].append(nodes.columns.get_loc(row["source"]))
    graph[1].append(nodes.columns.get_loc(row["target"]))

type_dict = {t:i for i, t in enumerate(nodes["cancer_type_detailed"].unique())}
type_dict

for i, k in enumerate(nodes["cancer_type_detailed"]):
  nodes["cancer_type_detailed"][i] = [type_dict[nodes["cancer_type_detailed"][i]]]

from datasets import Dataset, DatasetDict
custom_datadict = {"node_feat" : [[[x] for x in sublist] for sublist in nodes.iloc[:, :-3].values.tolist()], "num_nodes" : [nodes.shape[1] - 3] * len(nodes), 'y': nodes["cancer_type_detailed"], 'edge_index': [graph] * len(nodes), 'edge_attr': [[[1] for i in range(len(graph[0]))] for _ in range(len(nodes))]}
custom_dataset = Dataset.from_dict(custom_datadict)
custom_dataset = DatasetDict({"train": custom_dataset})
custom_dataset

# custom_dataset["train"][0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


DatasetDict({
    train: Dataset({
        features: ['node_feat', 'num_nodes', 'y', 'edge_index', 'edge_attr'],
        num_rows: 32
    })
})

In [4]:
custom_dataset["train"].features

{'node_feat': Sequence(feature=Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None), length=-1, id=None),
 'num_nodes': Value(dtype='int64', id=None),
 'y': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'edge_index': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'edge_attr': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None)}

In [5]:
from transformers.models.graphormer.collating_graphormer import preprocess_item, GraphormerDataCollator

dataset_processed = custom_dataset.map(preprocess_item, batched=False, writer_batch_size=50)

# split up training into training + validation
train_ds = dataset_processed['train']
print(train_ds[0].keys())

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

dict_keys(['node_feat', 'num_nodes', 'y', 'edge_index', 'edge_attr', 'input_nodes', 'attn_bias', 'attn_edge_type', 'spatial_pos', 'in_degree', 'out_degree', 'input_edges', 'labels'])


In [6]:
import torch

# torch.save(train_ds, 'breast_train.pt')

In [7]:
from datasets import load_metric

metric = load_metric("accuracy")
device = torch.device('cuda:0')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(labels)
    print(len(predictions[1]))
    print(predictions[0].shape)
    print(predictions)
    predictions = np.argmax(predictions, axis=0)
    return metric.compute(predictions=predictions, references=labels)

from transformers import GraphormerForGraphClassification

model_checkpoint = "clefourrier/graphormer-base-pcqm4mv2" # pre-trained model from which to fine-tune

model = GraphormerForGraphClassification.from_pretrained(
    model_checkpoint,
    num_classes=len(type_dict),
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
).to(device)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until
Some weights of GraphormerForGraphClassification were not initialized from the model checkpoint at clefourrier/graphormer-base-pcqm4mv2 and are newly initialized because the shapes did not match:
- classifier.classifier.weight: found shape torch.Size([1, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    "graph-classification",
    logging_dir="graph-classification",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    auto_find_batch_size=True, # batch size can be changed automatically to prevent OOMs
    gradient_accumulation_steps=3,
    dataloader_num_workers=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=train_ds,
    data_collator=GraphormerDataCollator()
)

In [9]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/opt/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


RuntimeError: CUDA error: device-side assert triggered